In [2]:
# Import

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import *
from pyspark.sql import functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd
import string, re, json
import sparknlp

In [3]:
# Caricamento dati necessari
#
#
ITRetweet = "./ITRetweet.json"
ITNoRetweet = "./ITNoRetweet.json"

# Creazione dataFrame
df_ITRetweet = spark.read.format("json").option("inferSchema", "true").option("multiLine", "true").load(ITRetweet)
df_ITNoRetweet = spark.read.format("json").option("inferSchema", "true").option("multiLine", "true").load(ITNoRetweet)

In [4]:
# ITRetweet

df_ITRetweet.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- id_str: string (nullable = true)
 |-- retweeted_status: struct (nullable = true)
 |    |-- full_text: string (nullable = true)



In [5]:
# ITNoRetweet

df_ITNoRetweet.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- geo: string (nullable = true)
 |-- id_str: string (nullable = true)



In [6]:
# UNIONE DEI RISULTATI

df_Tweets = df_ITRetweet\
    .selectExpr("id_str", "retweeted_status.full_text as full_text")\
    .union(df_ITNoRetweet.select("id_str", "full_text"))


df_Tweets = df_Tweets.select("full_text").distinct()
df_Tweets.count()

1086

In [7]:
# Definizione funzione per l'eliminazione dei caratteri speciali
#
#

def remove_punct(text):
    url_re = re.compile('https?://(www.)?\w+\.\w+(/\w+)*/?')
    punc_re = re.compile('[%s]' % re.escape(string.punctuation))
    mention_re = re.compile('@(\w+)')
    special = re.compile('[\$#,@&%£!=°§*/;-]')
    num_re = re.compile('( \\d+)')
    text = url_re.sub("", text)
    text = punc_re.sub("", text)
    text = mention_re.sub("", text)
    text = special.sub("", text)
    text = num_re.sub("", text)
    return text

# setup pyspark udf function
remove_features_udf = udf(lambda x: remove_punct(x), StringType())

In [1]:
# Eliminazione caratteri speciali
df_noHash = df_Tweets.withColumn('words_filtered',remove_features_udf("full_text"))

# Tokenizzazione
tkn = Tokenizer()\
      .setInputCol("words_filtered")\
      .setOutputCol("words")

# Eliminazione Stopwords
italianStopWords = StopWordsRemover.loadDefaultStopWords("italian")
stops = StopWordsRemover()\
        .setStopWords(italianStopWords)\
        .setInputCol("words")\
        .setOutputCol("words_nsw")

pipeline = Pipeline(stages = [tkn, stops])

df_TweetCleaned = pipeline.fit(df_noHash.select("words_filtered")).transform(df_noHash.select("words_filtered"))

df_TweetCleaned.show(10)

NameError: name 'df_Tweets' is not defined

In [ ]:
# SPARK NLP #